# Example for training Spiking CNN on subset of NMNIST digits

## The problem:
Training digit classifier(0-9) on a subset(1000 training and 100 testing) of NMNIST digit spikes recorded using DVS camera. Just chagne the training list to for full NMNIST training.

## Load proper paths for SLAYER Pytorch source modules

In [1]:
import sys, os
CURRENT_TEST_DIR = os.getcwd()
sys.path.append(CURRENT_TEST_DIR + "/slayerPytorch/src")

## Load required modules

SLAYER modules are available as `snn`
* The `spike-layer` module will be available as `snn.layer`.
* The `yaml-parameter` module will be availabe as `snn.params`.
* The `spike-loss` module will be available as `snn.loss`.
* The `spike-classifier` module will be available as `snn.predict`.
* The `spike-IO` module will be available as `snn.io`.


In [2]:
from datetime import datetime
import numpy as np
import tqdm.notebook as tqdm
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import slayerSNN as snn
from learningStats import learningStats
from IPython.display import HTML
import zipfile
from torch.utils.tensorboard import SummaryWriter

## Read SNN configuration from yaml file
See the file for all the configuration parameters. This configuration file will be used to describe the SNN. We will ignore the network configuration  describe in the yaml file here.

In [3]:
netParams = snn.params('network.yaml')

In [4]:
netParams['training']

{'error': {'type': 'NumSpikes',
  'probSlidingWin': 20,
  'tgtSpikeRegion': {'start': 0, 'stop': 300},
  'tgtSpikeCount': {True: 60, False: 10}},
 'path': {'in': '../data/NMNISTsmall/',
  'train': '../data/NMNISTsmall/train1K.txt',
  'test': '../data/NMNISTsmall/test100.txt'}}

## Extract NMNISTsmall dataset
This is a subset of NMNIST dataset containing first 1000 training samples and first 100 testing samples. The original NMNSIT dataset consists of full MNIST samples converted into spikes using DVS sensor moved in three repeatable saccadic motion. For details and full dataset download links, refer to [https://www.garrickorchard.com/datasets/n-mnist](https://www.garrickorchard.com/datasets/n-mnist)

In [5]:
with zipfile.ZipFile('../data/NMNISTsmall.zip') as zip_file:
    for member in zip_file.namelist():
        if not os.path.exists('./' + member):
            zip_file.extract(member, '../data/')

## Defne the dataset class
The dataset definition follows standard PyTorch dataset definition.
Internally, it utilizes snn.io modules to read spikes and returns the spike in correct tensor format (CHWT).
* `datasetPath`: the path where the spike files are stored.
* `sampleFile`: the file that contains a list of sample indices and its corresponding clases.
* `samplingTime`: the sampling time (in ms) to bin the spikes.
* `sampleLength`: the length of the sample (in ms)

Note: This is a simple dataset class. A dataset that utilizes the folder hierarchy or xml list is easy to create.

In [6]:
# Dataset definition
class nmnistDataset(Dataset):
    def __init__(self, datasetPath, sampleFile, samplingTime, sampleLength):
        self.path = datasetPath 
        self.samples = np.loadtxt(sampleFile).astype('int')
        self.samplingTime = samplingTime
        self.nTimeBins    = int(sampleLength / samplingTime)

    def __getitem__(self, index):
        inputIndex  = self.samples[index, 0]
        classLabel  = self.samples[index, 1]

        inputSpikes = snn.io.read2Dspikes(
                        self.path + str(inputIndex.item()) + '.bs2'
                        ).toSpikeTensor(torch.zeros((2,34,34,self.nTimeBins)),
                        samplingTime=self.samplingTime)
        desiredClass = torch.zeros((10, 1, 1, 1))
        desiredClass[classLabel,...] = 1
        return inputSpikes, desiredClass, classLabel

    def __len__(self):
        return self.samples.shape[0]

## Visualize the spike data

In [7]:
trainingSet = nmnistDataset(datasetPath =netParams['training']['path']['in'], 
                            sampleFile  =netParams['training']['path']['train'],
                            samplingTime=netParams['simulation']['Ts'],
                            sampleLength=netParams['simulation']['tSample'])

In [8]:
input, target, label = trainingSet[0]
anim = snn.io.animTD(snn.io.spikeArrayToEvent(input.reshape((2, 34, 34, -1)).cpu().data.numpy()))
HTML(anim.to_jshtml())

In [9]:
# Delete the rogue temp-file
try:
    os.remove('None0000000.png')
except FileNotFoundError:
    pass

## Define the network
The network definition follows similar style as standard PyTorch network definition, but it utilizes snn modules.

In [10]:
class PSPLayer(torch.nn.Module):
    def __init__(self, netParams=netParams):
        super(PSPLayer, self).__init__()
        self.slayer = snn.layer(netParams['neuron'], netParams['simulation'])
    def forward(self, x):
        return self.slayer.psp(x)

In [11]:
class ApplySpikeLayer(torch.nn.Module):
    def __init__(self, netParams=netParams):
        super(ApplySpikeLayer, self).__init__()
        self.slayer = snn.layer(netParams['neuron'], netParams['simulation'])
    def forward(self, x):
        return self.slayer.spike(x)

In [12]:
class SpikingNetwork(torch.nn.Module):
    def __init__(self, netParams=netParams):
        super(SpikingNetwork, self).__init__()
        # initialize slayer
        self.slayer = snn.layer(netParams['neuron'], netParams['simulation'])
        self.psplayer = PSPLayer(netParams)
        self.applyspikelayer = ApplySpikeLayer(netParams)
        # self.slayer = slayer
        
    def forward(self, spikeInput):
#         x = spikeInput
#         self.outputs[0] = spikeInput
#         for i, layer in enumerate(self.layers):
#             self.outputs[i+1] = self.slayer.spike(layer(self.slayer.psp(self.outputs[i])))
#         return self.outputs[-1]
        return self.layers.forward(spikeInput)
    
    def setLayers(self, layers):
        if isinstance(layers, list):
            self.layers = []
            for layer in layers:
                self.layers.append(self.psplayer)
                self.layers.append(layer)
                self.layers.append(self.applyspikelayer)
            self.layers = torch.nn.Sequential(*self.layers)
        else:
            raise Exception("layers should be a list of layers")
    def readyTraining(self):
        # Create network instance.
        # net = Network(netParams).to(device)
        # Split the network to run over multiple GPUs
        net = torch.nn.DataParallel(self.to(device), device_ids=deviceIds)
        
        # Create snn loss instance.
        error = snn.loss(netParams).to(device)

        # Define optimizer module.
        optimizer = torch.optim.Adam(net.parameters(), lr = 0.01, amsgrad = True)

        # Dataset and dataLoader instances.
        trainingSet = nmnistDataset(datasetPath =netParams['training']['path']['in'], 
                                    sampleFile  =netParams['training']['path']['train'],
                                    samplingTime=netParams['simulation']['Ts'],
                                    sampleLength=netParams['simulation']['tSample'])
        trainLoader = DataLoader(dataset=trainingSet, batch_size=8, shuffle=True, num_workers=4)

        testingSet = nmnistDataset(datasetPath  =netParams['training']['path']['in'], 
                                    sampleFile  =netParams['training']['path']['test'],
                                    samplingTime=netParams['simulation']['Ts'],
                                    sampleLength=netParams['simulation']['tSample'])
        testLoader = DataLoader(dataset=testingSet, batch_size=8, shuffle=True, num_workers=4)

        # Learning stats instance.
        stats = learningStats()
        
        self.netVars = {
            'net': net,
            'error': error,
            'optimizer': optimizer,
            'trainingSet': trainingSet,
            'trainLoader': trainLoader,
            'testingSet': testingSet,
            'testLoader': testLoader,
            'stats': stats
        }

In [13]:
class SpikingNetwork2(torch.nn.Module):
    def __init__(self, netParams=netParams):
        super(SpikingNetwork2, self).__init__()
        # initialize slayer
#         self.slayer = snn.layer(netParams['neuron'], netParams['simulation'])
        # self.slayer = slayer
        slayer = snn.layer(netParams['neuron'], netParams['simulation'])
        self.slayer = slayer
        # define network functions
        self.conv1 = slayer.conv(2, 16, 5, padding=1)
        self.conv2 = slayer.conv(16, 32, 3, padding=1)
        self.conv3 = slayer.conv(32, 64, 3, padding=1)
        self.pool1 = slayer.pool(2)
        self.pool2 = slayer.pool(2)
        self.fc1   = slayer.dense((8, 8, 64), 10)

    def forward(self, spikeInput):
#         x = spikeInput
#         for layer in self.layers:
#             x = self.slayer.spike(layer(self.slayer.psp(x)))
#         return x
        spikeLayer1 = self.slayer.spike(self.conv1(self.slayer.psp(spikeInput ))) # 32, 32, 16
        spikeLayer2 = self.slayer.spike(self.pool1(self.slayer.psp(spikeLayer1))) # 16, 16, 16
        spikeLayer3 = self.slayer.spike(self.conv2(self.slayer.psp(spikeLayer2))) # 16, 16, 32
        spikeLayer4 = self.slayer.spike(self.pool2(self.slayer.psp(spikeLayer3))) #  8,  8, 32
        spikeLayer5 = self.slayer.spike(self.conv3(self.slayer.psp(spikeLayer4))) #  8,  8, 64
        spikeOut    = self.slayer.spike(self.fc1  (self.slayer.psp(spikeLayer5))) #  10

        return spikeOut
    
    def setLayers(self, layers):
        if isinstance(layers, list):
            self.layers = torch.nn.ModuleList(layers)
    
    def readyTraining(self):
        # Create network instance.
        # net = Network(netParams).to(device)
        # Split the network to run over multiple GPUs
        net = torch.nn.DataParallel(self.to(device), device_ids=deviceIds)
        
        # Create snn loss instance.
        error = snn.loss(netParams).to(device)

        # Define optimizer module.
        optimizer = torch.optim.Adam(net.parameters(), lr = 0.01, amsgrad = True)

        # Dataset and dataLoader instances.
        trainingSet = nmnistDataset(datasetPath =netParams['training']['path']['in'], 
                                    sampleFile  =netParams['training']['path']['train'],
                                    samplingTime=netParams['simulation']['Ts'],
                                    sampleLength=netParams['simulation']['tSample'])
        trainLoader = DataLoader(dataset=trainingSet, batch_size=8, shuffle=True, num_workers=4)

        testingSet = nmnistDataset(datasetPath  =netParams['training']['path']['in'], 
                                    sampleFile  =netParams['training']['path']['test'],
                                    samplingTime=netParams['simulation']['Ts'],
                                    sampleLength=netParams['simulation']['tSample'])
        testLoader = DataLoader(dataset=testingSet, batch_size=8, shuffle=True, num_workers=4)

        # Learning stats instance.
        stats = learningStats()
        
        self.netVars = {
            'net': net,
            'error': error,
            'optimizer': optimizer,
            'trainingSet': trainingSet,
            'trainLoader': trainLoader,
            'testingSet': testingSet,
            'testLoader': testLoader,
            'stats': stats
        }

In [36]:
class SpikingNetwork3(torch.nn.Module):
    def __init__(self, netParams=netParams):
        super(SpikingNetwork3, self).__init__()
        # initialize slayer
#         self.slayer = snn.layer(netParams['neuron'], netParams['simulation'])
        # self.slayer = slayer
        slayer = snn.layer(netParams['neuron'], netParams['simulation'])
        self.slayer = slayer
        self.psplayer = PSPLayer(netParams)
        self.applyspikelayer = ApplySpikeLayer(netParams)
        # define network functions
        self.conv1 = slayer.conv(2, 16, 5, padding=1)
        self.conv2 = slayer.conv(16, 32, 3, padding=1)
        self.conv3 = slayer.conv(32, 64, 3, padding=1)
        self.pool1 = slayer.pool(2)
        self.pool2 = slayer.pool(2)
        self.fc1   = slayer.dense((8, 8, 64), 10)
        self.layers = torch.nn.Sequential(
            self.psplayer, self.conv1, self.applyspikelayer,
            self.psplayer, self.pool1, self.applyspikelayer,
            self.psplayer, self.conv2, self.applyspikelayer,
            self.psplayer, self.pool2, self.applyspikelayer,
            self.psplayer, self.conv3, self.applyspikelayer,
            self.psplayer, self.fc1, self.applyspikelayer,
        )

    def forward(self, spikeInput):
#         x = spikeInput
#         for layer in self.layers:
#             x = self.slayer.spike(layer(self.slayer.psp(x)))
#         return x
#         spikeLayer1 = self.applyspikelayer(self.conv1(self.psplayer(spikeInput ))) # 32, 32, 16
#         spikeLayer2 = self.applyspikelayer(self.pool1(self.psplayer(spikeLayer1))) # 16, 16, 16
#         spikeLayer3 = self.applyspikelayer(self.conv2(self.psplayer(spikeLayer2))) # 16, 16, 32
#         spikeLayer4 = self.applyspikelayer(self.pool2(self.psplayer(spikeLayer3))) #  8,  8, 32
#         spikeLayer5 = self.applyspikelayer(self.conv3(self.psplayer(spikeLayer4))) #  8,  8, 64
#         spikeOut    = self.applyspikelayer(self.fc1  (self.psplayer(spikeLayer5))) #  10

#         return spikeOut
        return self.layers(spikeInput)
    def setLayers(self, layers):
        if isinstance(layers, list):
            self.layers = torch.nn.ModuleList(layers)
    
    def readyTraining(self):
        # Create network instance.
        # net = Network(netParams).to(device)
        # Split the network to run over multiple GPUs
        net = torch.nn.DataParallel(self.to(device), device_ids=deviceIds)
        
        # Create snn loss instance.
        error = snn.loss(netParams).to(device)

        # Define optimizer module.
        optimizer = torch.optim.Adam(net.parameters(), lr = 0.01, amsgrad = True)

        # Dataset and dataLoader instances.
        trainingSet = nmnistDataset(datasetPath =netParams['training']['path']['in'], 
                                    sampleFile  =netParams['training']['path']['train'],
                                    samplingTime=netParams['simulation']['Ts'],
                                    sampleLength=netParams['simulation']['tSample'])
        trainLoader = DataLoader(dataset=trainingSet, batch_size=8, shuffle=True, num_workers=4)

        testingSet = nmnistDataset(datasetPath  =netParams['training']['path']['in'], 
                                    sampleFile  =netParams['training']['path']['test'],
                                    samplingTime=netParams['simulation']['Ts'],
                                    sampleLength=netParams['simulation']['tSample'])
        testLoader = DataLoader(dataset=testingSet, batch_size=8, shuffle=True, num_workers=4)

        # Learning stats instance.
        stats = learningStats()
        
        self.netVars = {
            'net': net,
            'error': error,
            'optimizer': optimizer,
            'trainingSet': trainingSet,
            'trainLoader': trainLoader,
            'testingSet': testingSet,
            'testLoader': testLoader,
            'stats': stats
        }

In [37]:
Network0 = SpikingNetwork3()
# slayer = Network0.slayer
# Network0.setLayers([
#             slayer.conv(2, 16, 5, padding=1),
#             slayer.conv(16, 32, 3, padding=1),
#             slayer.conv(32, 64, 3, padding=1),
#             slayer.pool(2),
#             slayer.pool(2),
#             slayer.dense((8,8,64), 10)
#         ])

In [13]:
class Network(torch.nn.Module):
    def __init__(self, netParams):
        super(Network, self).__init__()
        # initialize slayer
        slayer = snn.layer(netParams['neuron'], netParams['simulation'])
        self.slayer = slayer
        # define network functions
        self.conv1 = slayer.conv(2, 16, 5, padding=1)
        self.conv2 = slayer.conv(16, 32, 3, padding=1)
        self.conv3 = slayer.conv(32, 64, 3, padding=1)
        self.pool1 = slayer.pool(2)
        self.pool2 = slayer.pool(2)
        self.fc1   = slayer.dense((8, 8, 64), 10)

    def forward(self, spikeInput):
        spikeLayer1 = self.slayer.spike(self.conv1(self.slayer.psp(spikeInput ))) # 32, 32, 16
        spikeLayer2 = self.slayer.spike(self.pool1(self.slayer.psp(spikeLayer1))) # 16, 16, 16
        spikeLayer3 = self.slayer.spike(self.conv2(self.slayer.psp(spikeLayer2))) # 16, 16, 32
        spikeLayer4 = self.slayer.spike(self.pool2(self.slayer.psp(spikeLayer3))) #  8,  8, 32
        spikeLayer5 = self.slayer.spike(self.conv3(self.slayer.psp(spikeLayer4))) #  8,  8, 64
        spikeOut    = self.slayer.spike(self.fc1  (self.slayer.psp(spikeLayer5))) #  10

        return spikeOut

In [14]:
Netork1 = Network(netParams)

In [12]:
Networks = []

In [22]:
Networks.append(SpikingNetwork())
slayer = Networks[0].slayer
Networks[0].setLayers([
            slayer.conv(2, 6, 5, padding=1),
            slayer.conv(6, 12, 3, padding=1),
            slayer.pool(4),
            slayer.dense((8,8,12), 10)
        ])

NameError: name 'Networks' is not defined

In [14]:
Networks.append(SpikingNetwork())
slayer = Networks[1].slayer
Networks[1].setLayers([
            slayer.conv(2, 4, 5, padding=1),
            slayer.conv(4, 8, 3, padding=1),
            slayer.pool(4),
            slayer.dense((8,8,8), 10)
        ])

In [15]:
Networks.append(SpikingNetwork())
slayer = Networks[2].slayer
Networks[2].setLayers([
            slayer.conv(2, 6, 5, padding=1),
            slayer.conv(6, 12, 3, padding=1),
            slayer.pool(8),
            slayer.dense((4,4,12), 10)
        ])

In [16]:
Networks.append(SpikingNetwork())
slayer = Networks[3].slayer
Networks[3].setLayers([
            slayer.conv(2, 6, 5, padding=1),
            slayer.conv(6, 12, 3, padding=1),
            slayer.conv(12, 10, 3, padding=1),
            slayer.pool(32),
            slayer.dense((1,1,10), 10)
        ])

## Initialize the network
* Define the device to run the code on.
* Create network instance.
* Create loss instance.
* Define optimizer module.
* Define training and testing dataloader.
* Cereate instance for learningStats.

In [38]:
# Define the cuda device to run the code on.
# device = torch.device('cuda')
# Use multiple GPU's if available
device = torch.device('cuda:0')#:2') # should be the first GPU of deviceIDs 
deviceIds = [0]#2, 3]



In [39]:
# Define the cuda device to run the code on.
# device = torch.device('cuda')
# Use multiple GPU's if available
# device = torch.device('cuda:0')#2') # should be the first GPU of deviceIDs
# deviceIds = [0]#[2, 3]

# Create network instance.
# net = Network(netParams).to(device)
# Split the network to run over multiple GPUs
net = torch.nn.DataParallel(Network0.to(device), device_ids=deviceIds)

# Create snn loss instance.
error = snn.loss(netParams).to(device)

# Define optimizer module.
optimizer = torch.optim.Adam(net.parameters(), lr = 0.01, amsgrad = True)

# Dataset and dataLoader instances.
trainingSet = nmnistDataset(datasetPath =netParams['training']['path']['in'], 
                            sampleFile  =netParams['training']['path']['train'],
                            samplingTime=netParams['simulation']['Ts'],
                            sampleLength=netParams['simulation']['tSample'])
trainLoader = DataLoader(dataset=trainingSet, batch_size=8, shuffle=False, num_workers=4)

testingSet = nmnistDataset(datasetPath  =netParams['training']['path']['in'], 
                            sampleFile  =netParams['training']['path']['test'],
                            samplingTime=netParams['simulation']['Ts'],
                            sampleLength=netParams['simulation']['tSample'])
testLoader = DataLoader(dataset=testingSet, batch_size=8, shuffle=False, num_workers=4)

# Learning stats instance.
stats = learningStats()

# Train the network
Train the network for 100 epochs.

In [14]:
def train(Network, epochcount=1, writer=SummaryWriter()):
    for epoch in range(tqdm.trange(epochcount, desc='epoch')):
        # Reset training stats.
        stats = Network.netVars['stats']
        net = Network.netVars['net']
        trainLoader = Network.netVars['trainLoader']
        testLoader = Network.netVars['testLoader']
        error = Network.netVars['error']
        optimizer = Network.netVars['optimizer']
        
        stats.training.reset()
        tSt = datetime.now()

        # Training loop.
        for i, (input, target, label) in enumerate(tqdm.tqdm(trainLoader, desc='batch'), 0):
            # Move the input and target to correct GPU.
            input  = input.to(device)
            target = target.to(device) 

            # Forward pass of the network.
            output = net.forward(input)

            # Gather the training stats.
            stats.training.correctSamples += torch.sum( snn.predict.getClass(output) == label ).data.item()
            stats.training.numSamples     += len(label)

            # Calculate loss.
            loss = error.numSpikes(output, target)

            # Reset gradients to zero.
            optimizer.zero_grad()

            # Backward pass of the network.
            loss.backward()

            # Update weights.
            optimizer.step()

            # Gather training loss stats.
            stats.training.lossSum += loss.cpu().data.item()

            # Display training stats. (Suitable for normal python implementation)
            # stats.print(epoch, i, (datetime.now() - tSt).total_seconds())

        # Update training stats.
        stats.training.update()
        # Reset testing stats.
        stats.testing.reset()

        # Testing loop.
        # Same steps as Training loops except loss backpropagation and weight update.
        for i, (input, target, label) in enumerate(tqdm.trange(testLoader, desc='test'), 0):
            input  = input.to(device)
            target = target.to(device) 

            output = net.forward(input)

            stats.testing.correctSamples += torch.sum( snn.predict.getClass(output) == label ).data.item()
            stats.testing.numSamples     += len(label)

            loss = error.numSpikes(output, target)
            stats.testing.lossSum += loss.cpu().data.item()
            # stats.print(epoch, i)

        # Update testing stats.
        stats.testing.update()
        writer.add_scalar('Accuracy/train', stats.training.accuracy(), epoch)
        writer.add_scalar('Accuracy/test',  stats.testing.accuracy(),  epoch)
        writer.add_scalar('Loss/train',     stats.training.loss(),     epoch)
        writer.add_scalar('Loss/test',      stats.testing.loss(),      epoch)

        if epoch%10==0:  stats.print(epoch, timeElapsed=(datetime.now() - tSt).total_seconds())
    writer.close()

In [17]:
Network = Network0
Network.readyTraining()
stats = Network.netVars['stats']
net = Network.netVars['net']
trainLoader = Network.netVars['trainLoader']
testLoader = Network.netVars['testLoader']
error = Network.netVars['error']
optimizer = Network.netVars['optimizer']


In [40]:
for epoch in range(1):
    # Reset training stats.
    stats.training.reset()
    tSt = datetime.now()

    # Training loop.
    for i, (input, target, label) in enumerate(trainLoader, 0):
        # Move the input and target to correct GPU.
        input  = input.to(device)
        target = target.to(device) 

        # Forward pass of the network.
        %timeit output = net.forward(input)
        output = net.forward(input)

        # Gather the training stats.
        stats.training.correctSamples += torch.sum( snn.predict.getClass(output) == label ).data.item()
        stats.training.numSamples     += len(label)

        # Calculate loss.
        %timeit loss = error.numSpikes(output, target)
        loss = error.numSpikes(output, target)

        # Reset gradients to zero.
        optimizer.zero_grad()

        # Backward pass of the network.
        %timeit loss.backward(retain_graph=True)

        # Update weights.
        optimizer.step()

        # Gather training loss stats.
        stats.training.lossSum += loss.cpu().data.item()

        # Display training stats. (Suitable for normal python implementation)
        # stats.print(epoch, i, (datetime.now() - tSt).total_seconds())

    # Update training stats.
    stats.training.update()
    # Reset testing stats.
    stats.testing.reset()

    # Testing loop.
    # Same steps as Training loops except loss backpropagation and weight update.
    for i, (input, target, label) in enumerate(testLoader, 0):
        input  = input.to(device)
        target = target.to(device) 

        output = net.forward(input)

        stats.testing.correctSamples += torch.sum( snn.predict.getClass(output) == label ).data.item()
        stats.testing.numSamples     += len(label)

        loss = error.numSpikes(output, target)
        stats.testing.lossSum += loss.cpu().data.item()
        # stats.print(epoch, i)

    # Update testing stats.
    stats.testing.update()
    if epoch%10==0:  stats.print(epoch, timeElapsed=(datetime.now() - tSt).total_seconds())

438 ms ± 484 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
404 µs ± 2.35 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
286 ms ± 299 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Traceback (most recent call last):
  File "/home/rrs/anaconda3/envs/snn3/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/rrs/anaconda3/envs/snn3/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/rrs/anaconda3/envs/snn3/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/home/rrs/anaconda3/envs/snn3/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


KeyboardInterrupt: 

In [19]:
for i, Network in enumerate(Networks):
    Network.readyTraining()
    train(Network, epochcount=100, writer=SummaryWriter("modelset2/"+str(i), comment='model'+str(i)))


Epoch :          0,   60368.2810 ms elapsed
loss = 14.496      (min = 14.496     )    accuracy = 0.102    (max = 0.102   )
loss = 6.8815      (min = 6.8815     )    accuracy = 0.15     (max = 0.15    )

Epoch :         10,   59395.6040 ms elapsed
loss = 6.8892      (min = 6.8892     )    accuracy = 0.119    (max = 0.124   )
loss = 6.7959      (min = 6.7851     )    accuracy = 0.18     (max = 0.19    )

Epoch :         20,   58996.2950 ms elapsed
loss = 5.9286      (min = 5.9286     )    accuracy = 0.265    (max = 0.265   )
loss = 5.9972      (min = 5.9972     )    accuracy = 0.3      (max = 0.3     )

Epoch :         30,   58461.7180 ms elapsed
loss = 4.6313      (min = 4.6313     )    accuracy = 0.434    (max = 0.437   )
loss = 5.2892      (min = 5.2892     )    accuracy = 0.34     (max = 0.35    )

Epoch :         40,   58112.6710 ms elapsed
loss = 2.9551      (min = 2.9551     )    accuracy = 0.702    (max = 0.702   )
loss = 3.8736      (min = 3.8736     )    accuracy = 0.55     (m

## ~~Plot the Results~~